In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import kampuan as kp

kp.test()

'Hello World!'

In [3]:

phrase='สวัสดี'
kp.tokenize(phrase)

['สวัส', 'ดี']

In [4]:
kp.tokenize('ตลาด')

['ตลาด']

* 2 word 2 syllabus case


In [5]:
case_1 ={'กินข้าว':'กาวขิ้น',
         'หิวจัง':'หังจิว',
        'คำผวน':'ควนผำ',
        'นอนแล้ว':'แนวร้อน',
        'ตะปู':'ตูปะ',
        'นักเรียน':'เนียนรัก',
        'ขนม':'ขมหน่ะ'}

tmp=[kp.tokenize(key) for key in case_1.keys()]
tmp

# get 

[['กิน', 'ข้าว'],
 ['หิว', 'จัง'],
 ['คำ', 'ผวน'],
 ['นอน', 'แล้ว'],
 ['ตะ', 'ปู'],
 ['นัก', 'เรียน'],
 ['ขนม']]

# Subwords

* initial_consonant
    * single อักษรเดี่ยว
        * sound_level (low/mid/high)
        * low_single, low_double
    * cluster อักษรควบ
        * true_cluster อักษรควบแท้
        * fasle_cluster อักษรควบไม่แท้
    * leading_cluster อักษรนำ–อักษรตาม
        * hornum_leading หอ นำ
        * oarnum_leading ออ นำ
        * non_conform_leading นำด้วยสูง หรือ กลาง  นคร ขนม
* final_consonant
    * type (dead/live)
    * sound (eng)
        * k: กก
        * ng: กง
        * t: กด
        * m: กม
        * n: กน
        * p: กบ
        * y: เกย
        * v: เกอว
    * sound_level low/mid/high
* vowel
* tone http://www.thai-language.com/ref/tone-rules
    * tone_mark
    * tone_sound

In [53]:
import re
from kampuan.lang_tools import get_vowel_pattern,get_vowel_form,remove_tone_mark

vowel_pattern=get_vowel_pattern()
def match_pattern(pattern,text):
    m=re.match(pattern,text)
    return m

def match_vowel_form(text,vowel_patterns):
    for i,pattern in enumerate(vowel_patterns):
        m=re.match(pattern,text)
        if m:
            return (i,m,pattern)
    
    return (-1,text,text)
        
vowel_forms=get_vowel_form(REP='-')
test_text =['เขียว',
            'เกรียน',
            'ตู่',
            'ตด',
            'กร',
            'นอน',
           'ขนุน',
           'จัง',
           'กำ',
           'ใหญ่',
            'ใคร',
           'เดา',
           'เปรต',
           'เสร็จ',
           'ปีน',
           'เปี๊ยะ',
           'คั่ว',
           'แก้ม',
            'แช่ง',
            'แกล้ม',
            'แพร่',
           'แข็ง',
           'เกลอ',
           'เทอญ',
           'เกิด',
           'เตลิด',
           'กลม',
            'เลว',
            'เสก',
           'วัว',
           'เยี่ยม',
           'เพลง',
           'ปลอม',
           'หอม',
           'เวร',
           'อย่าง',
           'อยู่',
           'อย่า',
           'กวน' ,
           'แวว',
           'วน',
           'ว่าย',
           'สวย',
           'เขย']

for text in test_text:
    print(text)
    plain=remove_tone_mark(text)
    (i,m,pattern)=match_vowel_form(plain,vowel_pattern)   
    print('===')
    print(m.groups(),vowel_forms[i],pattern)

เขียว
===
('เ', 'ข', 'ีย', 'ว') เ-ีย (เ)(.{1,3})(ีย)([ก-ฮ]*)
เกรียน
===
('เ', 'กร', 'ีย', 'น') เ-ีย (เ)(.{1,3})(ีย)([ก-ฮ]*)
ตู่
===
('ต', 'ู', '') -ู (.{1,3})(ู)([ก-ฮ]*)
ตด
===
('ต', 'ด') - (.{1,2})([ก-ฮ])
กร
===
('ก', 'ร') - (.{1,2})([ก-ฮ])
นอน
===
('น', 'อ', 'น') -อ (.{1,3})(อ)([ก-ฮ]*)
ขนุน
===
('ขน', 'ุ', 'น') -ุ (.{1,3})(ุ)([ก-ฮ]*)
จัง
===
('จ', 'ั', 'ง') -ั (.{1,3})(ั)([ก-ฮ]*)
กำ
===
('ก', 'ำ', '') -ำ (.{1,3})(ำ)([ก-ฮ]*)
ใหญ่
===
('ใ', 'หญ', '') ใ- (ใ)(.{1,3})([ก-ฮ]*)
ใคร
===
('ใ', 'คร', '') ใ- (ใ)(.{1,3})([ก-ฮ]*)
เดา
===
('เ', 'ด', 'า', '') เ-า (เ)(.{1,3})(า)([ก-ฮ]*)
เปรต
===
('เ', 'ปรต', '') เ- (เ)(.{1,3})([ก-ฮ]*)
เสร็จ
===
('เ', 'สร', '็', 'จ') เ-็ (เ)(.{1,3})(็)([ก-ฮ]*)
ปีน
===
('ป', 'ี', 'น') -ี (.{1,3})(ี)([ก-ฮ]*)
เปี๊ยะ
===
('เ', 'ป', 'ียะ', '') เ-ียะ (เ)(.{1,3})(ียะ)([ก-ฮ]*)
คั่ว
===
('ค', 'ัว', '') -ัว (.{1,3})(ัว)([ก-ฮ]*)
แก้ม
===
('แ', 'กม', '') แ- (แ)(.{1,3})([ก-ฮ]*)
แช่ง
===
('แ', 'ชง', '') แ- (แ)(.{1,3})([ก-ฮ]*)
แกล้ม
===
('แ', 'กลม', '') แ- (แ)(.{1,3})([ก-ฮ]*)
แพร่


In [54]:
from kampuan.const import THAI_VOW

len(THAI_VOW)

20

In [55]:
# case เ แ need to verify ควบกล้ำ 
# 1 ตัว -> no สะกด
# 2 ตัว -> 1 สะกด
# 3 ตัว -> 1 สะกด

# อักษรนำ

# #initialize the special character cases
# #คำควบแท้
# self.two_char_combine = ['กร','กล', 'กว', 'คร', 'ขร', 'คล', 'ขล','คว', 'ขว', 'ตร', 'ปร', 'ปล', 'พร', 'พล' ,'ผล',
#            'บร','บล','ดร','ฟร','ฟล','ทร','จร','ซร','ปร','สร']
# #คำนำ
# self.lead_char_nosound = ['อย','หง','หญ','หน','หม','หย', 'หร','หล','หว']
# #อักษรสูงนำอักษรต่ำ
# self._lead_char_high_low = ['ขน', 'ขม','สม','สย','สน','ขย','ฝร','ถล','ผว','ตน','จม','ตล',]
# #อักษรสูงนำอักษรกลาง
# self._high_char_high_medium = ['ผท','ผด','ผก','ผอ','ผช']

In [56]:
import pythainlp
pythainlp.thai_tonemarks

'่้๊๋'

In [57]:
pythainlp.thai_punctuations

'๏๚๛'

In [58]:
['-'+char for char in pythainlp.thai_below_vowels]

['-ุ', '-ู']

In [59]:
['_'+char for char in pythainlp.thai_tonemarks]

['_่', '_้', '_๊', '_๋']

In [60]:
['_'+char for char in pythainlp.thai_above_vowels]

['_ั', '_ิ', '_ี', '_ึ', '_ื', '_ํ', '_็']

In [61]:
[char+'_' for char in pythainlp.thai_lead_vowels]

['เ_', 'แ_', 'โ_', 'ใ_', 'ไ_']

In [62]:
['_'+char for char in pythainlp.thai_follow_vowels]

['_ะ', '_า', '_ำ', '_ๅ']

In [63]:
pythainlp.thai_vowels

'ฤฦะัาำิีึืุูเแโใไๅํ็'

In [64]:
from typing import NamedTuple, Optional,List
class TestWord(NamedTuple):
    raw: str
    vowel_form: Optional[str]=None
    tone_mark: Optional[str]=None
    all_consonants: Optional[List[str]]=None


a = TestWord('เขียว','เ-ีย',None,['ข','ย','ว'])

In [65]:
test_list=[
TestWord('เขียว', 'เ-ีย', None, ['ข', 'ย', 'ว']),
TestWord('เขียว', 'เ-ีย', None, ['ข', 'ย', 'ว']),
TestWord('เขียว', 'เ-ีย'),
TestWord('เขียว', 'เ-ีย', None, ['ข', 'ย', 'ว']),
]


In [69]:
import pythainlp
from dataclasses import dataclass
from kampuan.sub_word import ThaiSubWord
test_words=[ThaiSubWord('เกรง')
,ThaiSubWord('แปลง')
,ThaiSubWord('กรง')
,ThaiSubWord('ธง')
,ThaiSubWord('ครี่')
,ThaiSubWord('แซง')
,ThaiSubWord('แปล')
,ThaiSubWord('แทรก')
,ThaiSubWord('ท้อ')
,ThaiSubWord('ท้อง')
,ThaiSubWord('ใคร')     
,ThaiSubWord('เสวย')   
,ThaiSubWord('เยี่ยม')
,ThaiSubWord('บรรณ')
,ThaiSubWord('สมุทร')
,ThaiSubWord('จันทร์')
,ThaiSubWord('พันธุ์') 
,ThaiSubWord('กริบ') 
,ThaiSubWord('อยาก')     
,ThaiSubWord('กบ') 
,ThaiSubWord('เกษม') 
           ]

# a=ThaiSubWord('เกลอ')
for a in test_words:
    print('----')
    print(a._raw)
    print(a._vowel_form_tup)
    print(a._con_split)
    print(a._true_con_tup)

check this case not sure
----
เกรง
[(0, 'เ')]
(2, 'ร')
[(1, 'ก'), (2, 'ร'), (3, 'ง')]
----
แปลง
[(0, 'แ')]
(2, 'ล')
[(1, 'ป'), (2, 'ล'), (3, 'ง')]
----
กรง
[]
(1, 'ร')
[(0, 'ก'), (1, 'ร'), (2, 'ง')]
----
ธง
[]
(0, 'ธ')
[(0, 'ธ'), (1, 'ง')]
----
ครี่
[(2, 'ี')]
(3, '่')
[(0, 'ค'), (1, 'ร')]
----
แซง
[(0, 'แ')]
(1, 'ซ')
[(1, 'ซ'), (2, 'ง')]
----
แปล
[(0, 'แ')]
(2, 'ล')
[(1, 'ป'), (2, 'ล')]
----
แทรก
[(0, 'แ')]
(2, 'ร')
[(1, 'ท'), (2, 'ร'), (3, 'ก')]
----
ท้อ
[(2, 'อ')]
(1, '้')
[(0, 'ท')]
----
ท้อง
[(2, 'อ')]
(1, '้')
[(0, 'ท'), (3, 'ง')]
----
ใคร
[(0, 'ใ')]
(2, 'ร')
[(1, 'ค'), (2, 'ร')]
----
เสวย
[(0, 'เ'), (2, 'ว')]
(2, 'ว')
[(1, 'ส'), (3, 'ย')]
----
เยี่ยม
[(0, 'เ'), (2, 'ี'), (4, 'ย')]
(3, '่')
[(1, 'ย'), (5, 'ม')]
----
บรรณ
[]
(0, 'บ')
[(0, 'บ'), (1, 'ร'), (2, 'ร'), (3, 'ณ')]
----
สมุทร
[(2, 'ุ')]
(2, 'ุ')
[(0, 'ส'), (1, 'ม'), (3, 'ท'), (4, 'ร')]
----
จันทร์
[(1, 'ั')]
(1, 'ั')
[(0, 'จ'), (2, 'น')]
----
พันธุ์
[(1, 'ั'), (4, 'ุ')]
(4, 'ุ')
[(0, 'พ'), (2, 'น')]
----
กริบ
[(2, 'ิ')]
(

In [70]:
a.__dict__

{'_raw': 'เกษม',
 '_vowels_tup': [(0, 'เ')],
 '_vowels': ['เ'],
 '_consonants_tup': [(1, 'ก'), (2, 'ษ'), (3, 'ม')],
 '_consonants': ['ก', 'ษ', 'ม'],
 '_tone_mark': [],
 '_mute_mark': -1,
 '_two_syllable': True,
 '_ex_vw_form': 'เ-',
 '_ex_regex': <re.Match object; span=(0, 4), match='เกษม'>,
 '_ex_pattern': '(เ)(.{1,3})([ก-ฮ]*)',
 '_vowel_form': 'เ-',
 '_vowel_con_tup': (-1, None),
 '_vowel_form_tup': [(0, 'เ')],
 '_mute_con_tup': [],
 '_mute_con': [],
 '_true_con_tup': [(1, 'ก'), (2, 'ษ'), (3, 'ม')],
 '_con_split': (2, 'ษ'),
 '_init_con_tup': [(1, 'ก'), (2, 'ษ')],
 '_final_con_tup': [(3, 'ม')],
 '_tone': None}

In [71]:
a._con_split

(2, 'ษ')

In [72]:
a._true_con_tup

[(1, 'ก'), (2, 'ษ'), (3, 'ม')]

In [73]:
ThaiSubWord('เอื้อง')._ex_regex.groups()

('เ', 'อ', 'ือ', 'ง')

In [74]:
ThaiSubWord('เอื้อง').__dict__

{'_raw': 'เอื้อง',
 '_vowels_tup': [(0, 'เ'), (2, 'ื')],
 '_vowels': ['เ', 'ื'],
 '_consonants_tup': [(1, 'อ'), (4, 'อ'), (5, 'ง')],
 '_consonants': ['อ', 'อ', 'ง'],
 '_tone_mark': ['้'],
 '_mute_mark': -1,
 '_two_syllable': False,
 '_ex_vw_form': 'เ-ือ',
 '_ex_regex': <re.Match object; span=(0, 5), match='เอือง'>,
 '_ex_pattern': '(เ)(.{1,3})(ือ)([ก-ฮ]*)',
 '_vowel_form': 'เ-ือ',
 '_vowel_con_tup': (4, 'อ'),
 '_vowel_form_tup': [(0, 'เ'), (2, 'ื'), (4, 'อ')],
 '_mute_con_tup': [],
 '_mute_con': [],
 '_true_con_tup': [(1, 'อ'), (5, 'ง')],
 '_con_split': (3, '้'),
 '_init_con_tup': [(1, 'อ')],
 '_final_con_tup': [(5, 'ง')],
 '_tone': None}

In [75]:
ThaiSubWord('เกรง').__dict__

{'_raw': 'เกรง',
 '_vowels_tup': [(0, 'เ')],
 '_vowels': ['เ'],
 '_consonants_tup': [(1, 'ก'), (2, 'ร'), (3, 'ง')],
 '_consonants': ['ก', 'ร', 'ง'],
 '_tone_mark': [],
 '_mute_mark': -1,
 '_two_syllable': False,
 '_ex_vw_form': 'เ-',
 '_ex_regex': <re.Match object; span=(0, 4), match='เกรง'>,
 '_ex_pattern': '(เ)(.{1,3})([ก-ฮ]*)',
 '_vowel_form': 'เ-',
 '_vowel_con_tup': (-1, None),
 '_vowel_form_tup': [(0, 'เ')],
 '_mute_con_tup': [],
 '_mute_con': [],
 '_true_con_tup': [(1, 'ก'), (2, 'ร'), (3, 'ง')],
 '_con_split': (2, 'ร'),
 '_init_con_tup': [(1, 'ก'), (2, 'ร')],
 '_final_con_tup': [(3, 'ง')],
 '_tone': None}

In [76]:
ThaiSubWord('เปรลี้ยง').__dict__

{'_raw': 'เปรลี้ยง',
 '_vowels_tup': [(0, 'เ'), (4, 'ี')],
 '_vowels': ['เ', 'ี'],
 '_consonants_tup': [(1, 'ป'), (2, 'ร'), (3, 'ล'), (6, 'ย'), (7, 'ง')],
 '_consonants': ['ป', 'ร', 'ล', 'ย', 'ง'],
 '_tone_mark': ['้'],
 '_mute_mark': -1,
 '_two_syllable': False,
 '_ex_vw_form': 'เ-ีย',
 '_ex_regex': <re.Match object; span=(0, 7), match='เปรลียง'>,
 '_ex_pattern': '(เ)(.{1,3})(ีย)([ก-ฮ]*)',
 '_vowel_form': 'เ-ีย',
 '_vowel_con_tup': (6, 'ย'),
 '_vowel_form_tup': [(0, 'เ'), (4, 'ี'), (6, 'ย')],
 '_mute_con_tup': [],
 '_mute_con': [],
 '_true_con_tup': [(1, 'ป'), (2, 'ร'), (3, 'ล'), (7, 'ง')],
 '_con_split': (5, '้'),
 '_init_con_tup': [(1, 'ป'), (2, 'ร'), (3, 'ล')],
 '_final_con_tup': [(7, 'ง')],
 '_tone': None}

In [79]:
for a in get_vowel_form('ปล'):
    ch=ThaiSubWord(a)
    print(ch._con_split,ch.vowel_form_tup)
    print(a ,':',ch._init_con_tup,ch._final_con_tup)
for a in get_vowel_form('กง'):
    ch=ThaiSubWord(a)
    print(ch._con_split,ch.vowel_form_tup)
    print(a ,':',ch._init_con_tup,ch._final_con_tup)

(4, 'ย') [(0, 'เ'), (3, 'ี'), (4, 'ย'), (5, 'ะ')]
เปลียะ : [(1, 'ป'), (2, 'ล')] []
(4, 'อ') [(0, 'เ'), (3, 'ื'), (4, 'อ'), (5, 'ะ')]
เปลือะ : [(1, 'ป'), (2, 'ล')] []
(4, 'ะ') [(0, 'เ'), (3, 'า'), (4, 'ะ')]
เปลาะ : [(1, 'ป'), (2, 'ล')] []
(4, 'ย') [(0, 'เ'), (3, 'ี'), (4, 'ย')]
เปลีย : [(1, 'ป'), (2, 'ล')] []
(4, 'อ') [(0, 'เ'), (3, 'ื'), (4, 'อ')]
เปลือ : [(1, 'ป'), (2, 'ล')] []
(3, 'ว') [(2, 'ั'), (3, 'ว'), (4, 'ะ')]
ปลัวะ : [(0, 'ป'), (1, 'ล')] []
(3, 'อ') [(0, 'เ'), (3, 'อ'), (4, 'ะ')]
เปลอะ : [(1, 'ป'), (2, 'ล')] []
(3, 'า') [(0, 'เ'), (3, 'า')]
เปลา : [(1, 'ป'), (2, 'ล')] []
(3, 'ะ') [(0, 'เ'), (3, 'ะ')]
เปละ : [(1, 'ป'), (2, 'ล')] []
(3, '็') [(0, 'เ'), (3, '็')]
เปล็ : [(1, 'ป'), (2, 'ล')] []
(3, 'ะ') [(0, 'แ'), (3, 'ะ')]
แปละ : [(1, 'ป'), (2, 'ล')] []
(3, '็') [(0, 'แ'), (3, '็')]
แปล็ : [(1, 'ป'), (2, 'ล')] []
(3, 'ะ') [(0, 'โ'), (3, 'ะ')]
โปละ : [(1, 'ป'), (2, 'ล')] []
(3, 'ิ') [(0, 'เ'), (3, 'ิ')]
เปลิ : [(1, 'ป'), (2, 'ล')] []
(3, 'อ') [(2, 'ื'), (3, 'อ')]
ปลือ : [(0, 'ป'),

In [122]:
df_vowel_form.values
vowel_list

NameError: name 'vowel_list' is not defined

In [93]:
a._ex

('เ', 'ย', 'ีย', 'ว')

In [130]:
df_vowel_form['รูปสระ'].str.replace(
        '\xa0', '').values

array(['-ะ', '-ั', '-ำ', 'ใ-', 'ไ-', 'ใ-ย', 'เ-า', '-า', ' -ิ', '-ี',
       '-ึ', '-ื', '-ือ', '-ุ', '-ู', 'เ-ะ', 'เ-็', 'เ-', 'แ-ะ', 'แ-็',
       'แ-', 'เ-ียะ', 'เ-ีย', 'เ-ือะ', 'เ-ือ', '-ัวะ', '-ัว', '-ว', 'โ-ะ',
       'โ-', 'เ-าะ', '-็อ', '-็', '-อ', 'เ-อะ', 'เ-ิ', 'เ-อ'],
      dtype=object)

In [145]:
a=[1,2,3,2,4,1]
b=[1,2]
c=a.copy()
for d in b:
    c.remove(d)
c

[3, 2, 4, 1]

In [146]:
a

[1, 2, 3, 2, 4, 1]

In [43]:
from kampuan.const import df_vowel_form

In [44]:
df_vowel_form

,รูปสระ
0,-ะ
1,-ั
2,-ำ
3,ใ-
4,ไ-
5,ใ-ย
6,เ-า
7,-า
8,-ิ
9,-ี


http://www.thai-language.com/ref/tone-rules

# Tien's

In [9]:
from kampuan.util.extract import thai_subwords

In [25]:
x=[thai_subwords(orig_word=syll) for syll in kp.puan_kam(phrase)]
x[0].start



'หว'

In [ ]:
class thai_match_pattern():
    def __init__(self, start_regex, vowel_regex, end_regex, sound_regex):
        self.start_regex = start_regex
        self.vowel_regex = vowel_regex
        self.end_regex = end_regex
        self.sound_regex = sound_regex
        
    def match_start(self, orig_word):        
        matches = re.findall(self.start_regex, orig_word, re.DOTALL)
        return matches
    

In [ ]:
#Reference https://www.geeksforgeeks.org/longest-common-substring-dp-29/
# Returns length of longest common  
# substring of X[0..m-1] and Y[0..n-1]  
def LCSubStr(X, Y, m, n): 
      
    # Create a table to store lengths of 
    # longest common suffixes of substrings.  
    # Note that LCSuff[i][j] contains the  
    # length of longest common suffix of  
    # X[0...i-1] and Y[0...j-1]. The first 
    # row and first column entries have no 
    # logical meaning, they are used only 
    # for simplicity of the program. 
      
    # LCSuff is the table with zero  
    # value initially in each cell 
    LCSuff = [[0 for k in range(n+1)] for l in range(m+1)] 
      
    # To store the length of  
    # longest common substring 
    result = 0 
  
    # Following steps to build 
    # LCSuff[m+1][n+1] in bottom up fashion 
    for i in range(m + 1): 
        for j in range(n + 1): 
            if (i == 0 or j == 0): 
                LCSuff[i][j] = 0
            elif (X[i-1] == Y[j-1]): 
                LCSuff[i][j] = LCSuff[i-1][j-1] + 1
                result = max(result, LCSuff[i][j]) 
            else: 
                LCSuff[i][j] = 0
    return result 

In [34]:
import re
class thai_subwords():
    """Design to take input as single Thai token or word and extract start, vowel, end and sound of said token
    For additional data regarding different Thai words types and pronunciations, 
    consult:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn"""
    def __init__(self, orig_word, debug=False):
        self.orig_word = orig_word #Try not to change this value
        self._start = ''
        self._vowel = ''
        self._end = ''
        self._sound = ''
        self._double_start_sound = False #used to flag with that has two starting alphabets with sound like 'ตลาด = ตะ ลาด'
        self._high_lead_low = False #case อักษรสูงนำอักษรต่ำ
        self.debug = debug
        
        #Need to deal with negative lookahead case for word like "เกรียน" to correctly extract กร
        self.vowels_form = ['ะ', ' ั' ' ็', 'า', ' ิ' , ' ่' , " ่" , ' ํ', ' ุ', ' ู', 'เ', 'ใ', 'ไ', 'โ', 'อ', 'ย', 'ว', 'ฤ', 'ฤๅ', 'ฦ', 'ฦๅ']
        combine_vowel_regex_1 = '(?!เ.{1,4}ี.*ย)(?!เ.{1,4}ี.*ยะ)(?!เ.{1,4}ื.*อ)(?!เ.{1,4}ื.*อะ)(?!เ.{1,4}.*อะ)(?!เ.{1,4}.*อ)(?!เ.{1,4}.*าะ)'
        combine_vowel_regex_2 = '(?!เ.{1,4}.*า)(?!แ.{1,4}.*า)(?!.{1,4}ัวะ)(?!โ.{1,4}.*ะ)'
        combine_vowel_regex = combine_vowel_regex_1 + combine_vowel_regex_2
        self.start_regex_1 = ['(.{1,4})ะ','(.{1,4})า', '(.{1,4})ิ.*', '(.{1,4})ี.*', '(.{1,4})ึ', '(.{1,4})ื.*', '(.{1,4})ุ.*', '(.{1,4})ู.*', 'เ(.{1,4})็', 
                            'เ(.{1,4}).*', '(.{1,4}).*อ','แ(.{1,4}).*', '(.{1,4})ั.*', 'โ(.{1,4}).*', '(.{1,4}).*ำ', 'ใ(.{1,4}).*', 'ไ(.{1,4})']
        self.start_regex_1  = [combine_vowel_regex+regex for regex in self.start_regex_1]
        self.start_regex_2 = ['เ(.{1,4})ี.*ยะ', 'เ(.{1,4})ี.*ย', 'เ(.{1,4})ื.*อะ', 'เ(.{1,4})ื.*อ', 'เ(.{1,4}).*อ', 'เ(.{1,4}).*อะ',
                              'เ(.{1,4}).*า',
                              'เ(.{1,4}).*าะ','แ(.{1,4}).*ะ','(.{1,4})ัวะ','โ(.{1,4}).*ะ', 'เ(.{1,4})็', 'เ(.{1,4})ิ.*']
        self.end_regex_1 = ['\[REP\]า(.{1,4})', '\[REP\]ิ(.{1,4})', '\[REP\]ี(.{1,4})', '\[REP\]ึ(.{1,4})', '\[REP\]ือ(.{1,4})', 
                            '\[REP\]ุ(.{1,4})', '\[REP\]ู(.{1,4})', 'เ\[REP\]็(.{1,4})', 'เ\[REP\](.{1,4})', '\[REP\]อ(.{1,4})',
                            'แ\[REP\](.{1,4})', '\[REP\]ั(.{1,4})', 'โ\[REP\](.{1,4})','\[REP\]ำ(.{1,4})', 'ใ\[REP\](.{1,4})', 
                            'ไ\[REP\](.{1,4})']
        self.end_regex_2 = ['เ\[REP\]ีย(.{1,4})', 'เ\[REP\]ือ(.{1,4})', 'เ\[REP\]อ(.{1,4})', 'เ\[REP\]อะ(.{1,4})','เ\[REP\]า(.{1,4})']
        
        self.start_regex  = self.start_regex_1 + self.start_regex_2 
        self.end_regex  = self.end_regex_1 + self.end_regex_2 
        #self.vowel_regex = vowel_regex
        #self.end_regex = end_regex
        self.sound_regex = ['(่)', '(้)', '.(๊)','()๋']
        
        #initialize the special character cases
        #คำควบแท้
        self.two_char_combine = ['กร','กล', 'กว', 'คร', 'ขร', 'คล', 'ขล','คว', 'ขว', 'ตร', 'ปร', 'ปล', 'พร', 'พล' ,'ผล',
                   'บร','บล','ดร','ฟร','ฟล','ทร','จร','ซร','ปร','สร']
        #คำนำ
        self.lead_char_nosound = ['อย','หง','หญ','หน','หม','หย', 'หร','หล','หว']
        #อักษรสูงนำอักษรต่ำ
        self._lead_char_high_low = ['ขน', 'ขม','สม','สย','สน','ขย','ฝร','ถล','ผว','ตน','จม','ตล',]
        #อักษรสูงนำอักษรกลาง
        self._high_char_high_medium = ['ผท','ผด','ผก','ผอ','ผช']
        
        
    # using property decorator 
    @property
    def start(self): 
        #print("getter method called") 
        temp_start = self.match_pattern(self.orig_word, self.start_regex)
        if len(temp_start)==1: return temp_start
        else: 
            return self.check_double_start_alphabets(temp_start)
        #return self.match_pattern(self.orig_word, self.start_regex)
    
    @property
    def sound(self):
        if self.match_pattern(self.orig_word, self.sound_regex) is None: return ''
        return self.match_pattern(self.orig_word, self.sound_regex)
    
    @property
    def vowel(self):
        return self.orig_word.replace(self.sound,'').replace(self.start,'[REP]',1) #[REP] will be used for conversion to Lu/Ru
#     def vowel(self):
#         return self.orig_word.replace(self.sound,'').replace(self.end,'').replace(self.start,'[REP]',1) #[REP] will be used for conversion to Lu/Ru
    
    @property #might not need this one in final application as we need vowel
    def end(self):
        if self.match_pattern(self.vowel, self.end_regex) not in self.vowels_form: 
            return self.match_pattern(self.vowel, self.end_regex)
        else:
            return None

#     # a setter function 
#     @start.setter 
#     def start(self): 
#         print("setter method called") 
#         self._start = match_pattern(self.start_regex)
    
    def match_pattern(self, text, regex_pattern):
        output_start = None
        if self.debug: print(text)
        for pattern in regex_pattern:
            matches = re.search(pattern, text, re.DOTALL)
            
            if matches is not None and self.debug: print(pattern, matches.group(1))
            if matches is not None: output_start = matches.group(1)
        return output_start
    
    def check_double_start_alphabets(self, text):
        '''Use to check in case starting alphabet is longer than one, 
        ref:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn'''
        #Function that check word in case like ขนม and else 
        
        two_char = self.lead_char_nosound + self.two_char_combine
        
        max_lcs = 0
        if text in two_char: return text
        else:
            for char in two_char:
                if LCSubStr(text, char, len(text), len(char)) > max_lcs: 
                    max_lcs = LCSubStr(text, char, len(text), len(char))
                    max_lcs_char = char
            return max_lcs_char
        
        return None
    
    def check_double_start_sound(self, text):
        '''Use to address the case where start alphabets are double and need to split to reflect actual pronunciation 
        ref:https://sites.google.com/site/phasathaionline/hnwy-kar-reiyn-ru2/phyaychnatn'''
        #Function that check word in case like ขนม and else 
        
        
        return None
    
    def remove_karan(self, regex_pattern):
        return None
        #Write specific function for removing ์